In [1]:
using Distributions, Random
using Plots
using JuMP
using HiGHS

In [2]:
# parametros
u  = 150;
q  = 25;
r_ =  5;
c  = 10;

## Sampling

In [4]:
Random.seed!(1);

dmin =  50;
dmax = 150;

nCenarios = 200;                                # Number of Scenarios
Ω = 1:nCenarios;                                    # Set of Scenarios
p = ones(nCenarios)*(1/nCenarios);                  # Equal Probability
ξ_true = rand(Uniform(dmin, dmax), nCenarios);

## a) True Problem

Solve decision problem with 200_000 samples

### Estimação

In [5]:
TrueProblem = Model(HiGHS.Optimizer);

# ========== Variáveis de Decisão ========== #
@variable(TrueProblem, x >= 0);
@variable(TrueProblem, yS[Ω] >= 0);
@variable(TrueProblem, yR[Ω] >= 0); 

In [6]:
# ========== Restrições ========== #
@constraint(TrueProblem, x <= u);
@constraint(TrueProblem, [ω in Ω], yS[ω] <= ξ_true[ω]);
@constraint(TrueProblem, [ω in Ω], yS[ω] + yR[ω] <= x);

In [7]:
# ========== Função Objetivo ========== #
@objective(TrueProblem, Max, sum(p[ω]*(q*yS[ω] + r_*yR[ω] - c*x) for ω in Ω));

optimize!(TrueProblem);

status_True      = termination_status(TrueProblem);

ObjFun_True      = JuMP.objective_value(TrueProblem);
xOpt_True        = JuMP.value(x);
ySOpt_True       = JuMP.value.(yS);
yROpt_True       = JuMP.value.(yR);

Presolve : Reductions: rows 200(-201); columns 401(-0); elements 600(-201)
INFO   : Solving the presolved LP
INFO   : Scaling: Matrix has [min, max] values of [1, 1] within [0.2, 5] so no scaling performed
INFO   : Basis condition estimate of           1 is within the tolerance of 1e+14
INFO   : Using dual simplex solver - serial
       Iteration        Objective     Infeasibilities num(sum)
DuPh2          0     3.2525405078e+03 Pr: 200(50017.1)
DuPh2         50     2.2034006353e+03 Pr: 150(35624.9)
DuPh2        100     1.6038479513e+03 Pr: 180(9279.88)
DuPh2        150     1.4935633915e+03 Pr: 130(4869.81)
DuPh2        200     1.4130128149e+03 Pr: 80(1648.74)
DuPh2        231     1.3126927332e+03 Pr: 0(0)
DuPh2        231     1.3124942344e+03 Pr: 0(0)
INFO   : Dual simplex iterations [Ph1 0; Ph2 231; Pr 0] Total 231
INFO   : Solving the original LP from the solution after postsolve
INFO   : Scaling: Matrix has [min, max] values of [1, 1] within [0.2, 5] so no scaling performed
INFO   

### Resultados

In [8]:
println("True Problem -- Number Samples: ", nCenarios, "\n");
println("     -> Status: ", status_True);
println("     -> Objective Function: ", ObjFun_True);
println("     -> First-Stage Cost: ", c*xOpt_True);
println("     -> First-Stage Solution: ", xOpt_True);

True Problem -- Number Samples: 200

     -> Status: OPTIMAL
     -> Objective Function: 1312.4942344258855
     -> First-Stage Cost: 1251.31263278617
     -> First-Stage Solution: 125.13126327861701


## b) Scenario Wise

Solve decision problem with 100 samples

### Estimação

In [16]:
TrueProblem = Model(HiGHS.Optimizer);

# ========== Variáveis de Decisão ========== #
@variable(TrueProblem, x >= 0);
@variable(TrueProblem, yS[Ω[1:100]] >= 0);
@variable(TrueProblem, yR[Ω[1:100]] >= 0); 

In [17]:
# ========== Restrições ========== #
@constraint(TrueProblem, x <= u);
@constraint(TrueProblem, [ω in Ω[1:100]], yS[ω] <= ξ_true[ω]);
@constraint(TrueProblem, [ω in Ω[1:100]], yS[ω] + yR[ω] <= x);

In [18]:
# ========== Função Objetivo ========== #
@objective(TrueProblem, Max, sum(p[ω]*(q*yS[ω] + r_*yR[ω] - c*x) for ω in Ω[1:100]));

optimize!(TrueProblem);

status_True      = termination_status(TrueProblem);

ObjFun_True      = JuMP.objective_value(TrueProblem);
xOpt_True        = JuMP.value(x);
ySOpt_True       = JuMP.value.(yS);
yROpt_True       = JuMP.value.(yR);

Presolve : Reductions: rows 100(-101); columns 201(-0); elements 300(-101)
INFO   : Solving the presolved LP
INFO   : Scaling: Matrix has [min, max] values of [1, 1] within [0.2, 5] so no scaling performed
INFO   : Basis condition estimate of           1 is within the tolerance of 1e+14
INFO   : Using dual simplex solver - serial
       Iteration        Objective     Infeasibilities num(sum)
DuPh2          0     1.6086341443e+03 Pr: 100(24868.3)
DuPh2         50     7.9381672431e+02 Pr: 52(6290.92)
DuPh2        100     6.7220041969e+02 Pr: 12(1114.9)
DuPh2        115     6.4479088417e+02 Pr: 0(0)
DuPh2        115     6.4474222054e+02 Pr: 0(0)
INFO   : Dual simplex iterations [Ph1 0; Ph2 115; Pr 0] Total 115
INFO   : Solving the original LP from the solution after postsolve
INFO   : Scaling: Matrix has [min, max] values of [1, 1] within [0.2, 5] so no scaling performed
INFO   : Basis condition estimate of   1.273e+04 is within the tolerance of 1e+14
Postsolve  : 0
Time       :     0.00


### Resultados

In [21]:
println("Scenario Wise -- Number Samples: ", nCenarios/2, "\n");
println("     -> Status: ", status_True);
println("     -> Objective Function: ", ObjFun_True);
println("     -> First-Stage Cost: ", c*xOpt_True);
println("     -> First-Stage Solution: ", xOpt_True);

Scenario Wise -- Number Samples: 100.0

     -> Status: OPTIMAL
     -> Objective Function: 644.7422205401214
     -> First-Stage Cost: 1202.5571355154577
     -> First-Stage Solution: 120.25571355154578


## c) Linear Decision Rule

### Estimação

In [24]:
TrueProblem = Model(HiGHS.Optimizer);

# ========== Variáveis de Decisão ========== #
@variable(TrueProblem, x >= 0);
# @variable(TrueProblem, yS[Ω] >= 0);
# @variable(TrueProblem, yR[Ω] >= 0); 
@variable(TrueProblem, lS);
@variable(TrueProblem, lR);
@variable(TrueProblem, l0S);
@variable(TrueProblem, l0R);

In [25]:
# ========== Restrições ========== #
@constraint(TrueProblem, x <= u);
@constraint(TrueProblem, [ω in Ω], lS*ξ_true[ω] + l0S <= ξ_true[ω]);
@constraint(TrueProblem, [ω in Ω], lS*ξ_true[ω] + l0S + lR*ξ_true[ω] + l0R <= x);
@constraint(TrueProblem, [ω in Ω], lS*ξ_true[ω] + l0S >= 0);
@constraint(TrueProblem, [ω in Ω], lR*ξ_true[ω] + l0R >= 0);

In [ ]:
# ========== Função Objetivo ========== #
@objective(TrueProblem, Max, sum(p[ω]*(q*(lS*ξ_true[ω] + l0S) + r_*(lR*ξ_true[ω] + l0R) - c*x) for ω in Ω));

optimize!(TrueProblem);

status_True      = termination_status(TrueProblem);

ObjFun_True      = JuMP.objective_value(TrueProblem);
xOpt_True        = JuMP.value(x);

In [45]:
# ySOpt_True       = 
# JuMP.value.(lS)*ξ_true[Ω] + JuMP.value.(l0S);

In [44]:
JuMP.value.(l0S)

-0.0

### Resultados

In [32]:
println("True Problem -- Number Samples: ", nCenarios, "\n");
println("     -> Status: ", status_True);
println("     -> Objective Function: ", ObjFun_True);
println("     -> First-Stage Cost: ", c*xOpt_True);
println("     -> First-Stage Solution: ", xOpt_True);

True Problem -- Number Samples: 200

     -> Status: OPTIMAL
     -> Objective Function: 1251.7597359355116
     -> First-Stage Cost: 1499.9046588986134
     -> First-Stage Solution: 149.99046588986135


## d) Piecewise Linear